# With LangChain

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Bedrock
from langchain.callbacks.base import BaseCallbackHandler

class CustomCallBackHandler(BaseCallbackHandler):

    # 아래 함수에서 Frontend 와 Socket으로 연결해서 실시간으로 UI로 전달.
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"My custom handler, token: {token}")


parameters = {
    "max_tokens_to_sample": 200,  # min:0, max:4096, default:200
    "stop_sequences": ["\n\nHuman:"],
    "temperature": 1,  # min:0, max:1, default:0.5
    "top_p": 1,  # min:0, max:1, default:1
    "top_k": 225,  # min:0, max:500, default:250
}

model_id = "anthropic.claude-v2:1"
llm = Bedrock(
    model_id=model_id,
    streaming=True,
    model_kwargs=parameters,
    callbacks=[CustomCallBackHandler()],
)

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(llm=llm, verbose=True)
conversation.predict(input="Hi there!")

# Without LangChain

In [ ]:
import boto3
import json

bedrock_region = "us-west-2"

def get_bedrock_client():
    return boto3.client(
        service_name='bedrock-runtime',
        region_name=bedrock_region,
    )

def get_stream_from_bedrock_client(model_id: str, parameters: dict):
    bedrock_client = get_bedrock_client()
    accept = 'application/json'
    content_type = 'application/json'
    return bedrock_client.invoke_model_with_response_stream(
        body=json.dumps(parameters),
        modelId=model_id, accept=accept, contentType=content_type
    )

def generate_stream_answer_with_context(question: str,
                                        max_tokens: int = 200, temperature: float = 0,
                                        top_p: float = 0.9,
                                        top_k: int = 250, model_id: str = 'anthropic.claude-v2:1'):
    prompt = "".join([f"\n\nHuman: {question}", "\n\nAssistant:"])

    parameters = {
        "prompt": prompt,
        "max_tokens_to_sample": int(max_tokens),  # min:0, max:8,000, default:200
        "stop_sequences": ["\n\nHuman:"],
        "temperature": float(temperature),  # min:0, max:1, default:0.5
        "top_p": float(top_p),  # min:0, max:1, default:1
        "top_k": int(top_k)  # min:0, max:500, default:250
    }

    bedrock_client = get_bedrock_client()
    accept = 'application/json'
    content_type = 'application/json'
    response = bedrock_client.invoke_model_with_response_stream(
        body=json.dumps(parameters),
        modelId=model_id, accept=accept, contentType=content_type
    )

    return response.get('body')

In [ ]:
question = "안녕 베드락에 대해서 설명해줘"

stream = generate_stream_answer_with_context(question="안녕?!")

output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']

            #아래 부분을 Socket 연결해서 실시간으로 UI 로 전달.
            print(text)